In [ ]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

2025-06-05 10:27:49.776345: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-05 10:27:49.976828: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-05 10:27:50.161251: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749099470.328399   38911 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749099470.361646   38911 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1749099470.679868   38911 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

In [2]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    fill_mode='nearest',
    validation_split=0.2  # Use validation split
)

test_datagen = ImageDataGenerator(rescale=1./255)


In [4]:
# Training data
x_train = train_datagen.flow_from_directory(
    "/home/manishji/SMT-Interactive-Learning/Dataset/train",  # Update this path
    target_size=(64, 64),
    batch_size=32,
    class_mode="categorical",
    subset='training'  # Use training subset
)

# Validation data from training directory
x_val = train_datagen.flow_from_directory(
    "/home/manishji/SMT-Interactive-Learning/Dataset/train",  # Update this path
    target_size=(64, 64),
    batch_size=32,
    class_mode="categorical",
    subset='validation'  # Use validation subset
)

# Test data
x_test = test_datagen.flow_from_directory(
    "/home/manishji/SMT-Interactive-Learning/Dataset/test",  # Update this path
    target_size=(64, 64),
    batch_size=32,
    class_mode="categorical"
)

print("Class indices:", x_train.class_indices)

Found 8754 images belonging to 3 classes.


Found 2187 images belonging to 3 classes.
Found 300 images belonging to 3 classes.
Class indices: {'circle': 0, 'square': 1, 'triangle': 2}


In [5]:
def create_improved_model():
    model = Sequential([
        # First Convolutional Block
        Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        
        # Second Convolutional Block
        Conv2D(64, (3, 3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        
        # Third Convolutional Block
        Conv2D(128, (3, 3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        
        # Flatten and Dense Layers
        Flatten(),
        Dropout(0.5),
        Dense(256, activation='relu'),
        BatchNormalization(),
        Dropout(0.3),
        Dense(128, activation='relu'),
        Dropout(0.2),
        Dense(3, activation='softmax')  # 3 classes: circle, square, triangle
    ])
    return model

# Create and compile the improved model
model = create_improved_model()
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print(model.summary())

/home/manishji/.venv/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025-06-05 10:32:53.097628: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 62, 62, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 62, 62, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 31, 31, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 29, 29, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 29, 29, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 12, 12, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 12, 12, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 6, 6, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 4608)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 4608)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │     1,179,904 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 3)              │           387 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,308,355 (4.99 MB)

 Trainable params: 1,307,395 (4.99 MB)

 Non-trainable params: 960 (3.75 KB)

None


In [6]:
# Define callbacks
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=5,
    min_lr=0.0001
)

# Train the model
history = model.fit(
    x_train,
    steps_per_epoch=len(x_train),
    epochs=50,  # Increased epochs with early stopping
    validation_data=x_val,
    validation_steps=len(x_val),
    callbacks=[early_stopping, reduce_lr],
    verbose=1
)

# Save the improved model
model.save("improved_shapes.h5")


/home/manishji/.venv/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50
274/274 ━━━━━━━━━━━━━━━━━━━━ 0s 216ms/step - accuracy: 0.7946 - loss: 0.4980

/home/manishji/.venv/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


274/274 ━━━━━━━━━━━━━━━━━━━━ 69s 239ms/step - accuracy: 0.7950 - loss: 0.4970 - val_accuracy: 0.5560 - val_loss: 1.4488 - learning_rate: 0.0010
Epoch 2/50
274/274 ━━━━━━━━━━━━━━━━━━━━ 53s 192ms/step - accuracy: 0.9902 - loss: 0.0292 - val_accuracy: 0.9835 - val_loss: 0.0560 - learning_rate: 0.0010
Epoch 3/50
274/274 ━━━━━━━━━━━━━━━━━━━━ 48s 177ms/step - accuracy: 0.9953 - loss: 0.0161 - val_accuracy: 0.9995 - val_loss: 0.0010 - learning_rate: 0.0010
Epoch 4/50
274/274 ━━━━━━━━━━━━━━━━━━━━ 48s 177ms/step - accuracy: 0.9983 - loss: 0.0060 - val_accuracy: 0.9991 - val_loss: 0.0021 - learning_rate: 0.0010
Epoch 5/50
274/274 ━━━━━━━━━━━━━━━━━━━━ 49s 177ms/step - accuracy: 0.9960 - loss: 0.0098 - val_accuracy: 1.0000 - val_loss: 2.5074e-04 - learning_rate: 0.0010
Epoch 6/50
274/274 ━━━━━━━━━━━━━━━━━━━━ 49s 177ms/step - accuracy: 0.9979 - loss: 0.0051 - val_accuracy: 0.9995 - val_loss: 7.3432e-04 - learning_rate: 0.0010
Epoch 7/50
274/274 ━━━━━━━━━━━━━━━━━━━━ 49s 177ms/step - accuracy: 0.9978

In [7]:
def predict_shape(image_path, model_path="improved_shapes.h5"):
    """
    Predict the shape from an image with proper preprocessing
    """
    # Load the model
    model = load_model(model_path)
    
    # Load and preprocess the image
    img = image.load_img(image_path, target_size=(64, 64))
    img_array = image.img_to_array(img)
    
    # IMPORTANT: Normalize the image (same as training)
    img_array = img_array / 255.0
    
    # Add batch dimension
    img_array = np.expand_dims(img_array, axis=0)
    
    # Make prediction
    predictions = model.predict(img_array)
    predicted_class = np.argmax(predictions)
    confidence = np.max(predictions)
    
    # Class labels (ensure this matches your training data)
    class_labels = ['circle', 'square', 'triangle']
    
    return {
        'predicted_class': class_labels[predicted_class],
        'confidence': float(confidence),
        'all_probabilities': {
            class_labels[i]: float(predictions[0][i]) 
            for i in range(len(class_labels))
        }
    }

In [8]:
def test_predictions():
    """Test the model with sample images"""
    
    # Test with different images
    test_images = [
        "/home/manishji/SMT-Interactive-Learning/Dataset/test/circle/0.png",    # Update these paths
        "/home/manishji/SMT-Interactive-Learning/Dataset/test/square/0.png",
        "/phome/manishji/SMT-Interactive-Learning/Dataset/test/triangle/0.png"
    ]
    
    for img_path in test_images:
        try:
            result = predict_shape(img_path)
            print(f"\nImage: {img_path}")
            print(f"Predicted: {result['predicted_class']}")
            print(f"Confidence: {result['confidence']:.4f}")
            print("All probabilities:")
            for class_name, prob in result['all_probabilities'].items():
                print(f"  {class_name}: {prob:.4f}")
        except Exception as e:
            print(f"Error processing {img_path}: {e}")

# Uncomment to test
test_predictions()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 199ms/step



Image: /home/manishji/SMT-Interactive-Learning/Dataset/test/circle/0.png
Predicted: circle
Confidence: 1.0000
All probabilities:
  circle: 1.0000
  square: 0.0000
  triangle: 0.0000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step

Image: /home/manishji/SMT-Interactive-Learning/Dataset/test/square/0.png
Predicted: square
Confidence: 1.0000
All probabilities:
  circle: 0.0000
  square: 1.0000
  triangle: 0.0000


Error processing /phome/manishji/SMT-Interactive-Learning/Dataset/test/triangle/0.png: [Errno 2] No such file or directory: '/phome/manishji/SMT-Interactive-Learning/Dataset/test/triangle/0.png'


In [ ]:
# Evaluate the model on test data
def evaluate_model():
    """Evaluate the model on test data"""
    model = load_model("improved_shapes.h5")
    
    # Evaluate on test data
    test_loss, test_accuracy = model.evaluate(x_test, verbose=1)
    print(f"\nTest Accuracy: {test_accuracy:.4f}")
    print(f"Test Loss: {test_loss:.4f}")
    
    return test_accuracy, test_loss

# Uncomment to evaluate
evaluate_model()